In [ ]:
! pip install datasets evaluate transformers transformers[torch] rouge-score nltk --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


In [ ]:
import transformers
from datasets import load_dataset, concatenate_datasets
from evaluate import load
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
from huggingface_hub import notebook_login
import numpy as np
import nltk

In [ ]:
notebook_login()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
torch.cuda.set_device(device)
torch.cuda.current_device()

0

In [ ]:
#datasets_tr = load_dataset("MoritzLaurer/multilingual-NLI-26lang-2mil7", split=["tr_anli", "tr_fever", "tr_ling", "tr_mnli", "tr_wanli"])

In [ ]:
dataset_files = {"tr_anli": "tr_anli.parquet",
                 "tr_fever": "tr_fever.parquet",
                 "tr_ling": "tr_ling.parquet",
                 "tr_mnli": "tr_mnli.parquet",
                 "tr_wanli": "tr_wanli.parquet"}
dict_tr = load_dataset("parquet", data_files=dataset_files)
datasets_tr = [dict_tr['tr_anli'], dict_tr['tr_fever'], dict_tr['tr_ling'], dict_tr['tr_mnli'], dict_tr['tr_wanli']]

Generating tr_anli split: 0 examples [00:00, ? examples/s]

Generating tr_fever split: 0 examples [00:00, ? examples/s]

Generating tr_ling split: 0 examples [00:00, ? examples/s]

Generating tr_mnli split: 0 examples [00:00, ? examples/s]

Generating tr_wanli split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = concatenate_datasets(datasets_tr)
dataset = dataset.remove_columns(["premise_original", "hypothesis_original"])

In [ ]:
dataset

Dataset({
    features: ['label', 'premise', 'hypothesis'],
    num_rows: 105000
})

In [ ]:
label_col = ['gereklilik' if x[1] == 0 else 'nötr' if x[1] == 1 else 'çelişki' for x in enumerate(dataset['label'])]

In [ ]:
dataset = dataset.rename_column("label", "labelnum")
dataset = dataset.add_column("labelstr", label_col)

In [ ]:
dataset_shuffled = dataset.shuffle(seed=42)
dataset_eval = dataset_shuffled.select(range(5000))
dataset_shuffled = dataset_shuffled.select(range(5000, len(dataset_shuffled)))
dataset_ranged = dataset_shuffled.select(range(20000))

In [ ]:
dataset_splitted = dataset_ranged.train_test_split(test_size=0.2, seed=42)

In [ ]:
print(dataset_splitted['train'].shape, dataset_splitted['test'].shape)

(16000, 4) (4000, 4)


In [ ]:
dataset_splitted['train'][0]

{'labelnum': 1,
 'premise': 'Planı düz sokakları ve büyük meydanları olan simetrik bir tasarımdı.',
 'hypothesis': 'Planı, düz sokakların basit bir tasarımını yaparak etrafta dolaşmayı kolaylaştırmaktı.',
 'labelstr': 'nötr'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("boun-tabi-LMG/turna_nli_nli_tr")

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def prompt_format(row):
  return f"hipotez: {row[1]} önerme: {row[0]}"

In [ ]:
max_input_length = 512
max_target_length = 1

def preprocess_function(examples):
  inputs = [prompt_format(x) for x in zip(examples['premise'], examples['hypothesis'])]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
  labels = tokenizer(text_target=examples["labelstr"], max_length=max_target_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
dataset_encoded = dataset_splitted.map(preprocess_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("boun-tabi-LMG/turna_nli_nli_tr")

config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
metric = load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
args = Seq2SeqTrainingArguments(
    "turna-nlitr-finetuned-nli",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    predict_with_generate=True,
    push_to_hub=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.833600,0.775732,35.475000,35.525000,35.575000,35.575000,2.452200
2,0.738100,0.779433,35.625000,35.650000,35.675000,35.675000,2.451800
3,0.801500,0.772226,35.625000,35.650000,35.675000,35.650000,2.453500
4,0.785000,0.774786,35.525000,35.550000,35.575000,35.550000,2.458000
5,0.778200,0.771111,35.662500,35.675000,35.700000,35.675000,2.455000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=20000, training_loss=0.7808587036132812, metrics={'train_runtime': 17157.5319, 'train_samples_per_second': 4.663, 'train_steps_per_second': 1.166, 'total_flos': 2.7252926447616e+17, 'train_loss': 0.7808587036132812, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/aniltepe/turna-nlitr-finetuned-nli/commit/0cdfa66ee74af5f540edb69b1f4a2303ec92773e', commit_message='End of training', commit_description='', oid='0cdfa66ee74af5f540edb69b1f4a2303ec92773e', pr_url=None, pr_revision=None, pr_num=None)